In [2]:
def get_prec_number(a_string):
    # this function gives the float part of a precipitation measurement
    final = a_string.strip()
    return float(final[0:4])
    

def get_prec_letter(a_string):
    # this function gives the letter representing the tyoe of precipitation measurement
    final = a_string.strip()
    return final[-1]

In [3]:
from pymongo import MongoClient
from pyspark.sql import SparkSession
client = MongoClient() # so we connect on the default host and port
db = client.fit3182_assignment_db # create a new database fit3182_assignment_db
the_collection = db.climate  # create a new collection called 'the_collection'
try:
    the_collection.drop()
except NameError:
    pass
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'
os.environ['PYSPARK_PYTHON'] = "/home/student/FIT5148/jupyter/bin/python"
spark = SparkSession.builder.config("spark.archives", "pyspark_conda_env.tar.gz#environment").getOrCreate()

from pprint import pprint
import ast
import sys
import time
import json
from pymongo import MongoClient
import pygeohash as pgh
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

def sendDataToDB(iter):
    client = MongoClient() # so we connect on the default host and port
    db = client.fit3182_assignment_db # create a new database fit3182_assignment_db
    the_collection = db.climate  # create a new collection called 'the_collection'
    climate_value = 0
    n_climate = []
    same_location = []
    for record in iter:  
        first_record = ast.literal_eval(record[1])
        print(first_record)
        if first_record[-1] == 'Producer1':
            climate_value = first_record
            
        else:
            n_climate.append(first_record)
    for record in n_climate:
        if (pgh.encode(float(record[0]), float(record[1]), precision= 3) 
        == pgh.encode(float(climate_value[0]), float(climate_value[1]), precision =3)):
            same_location.append(record)
            
    geo_dictionary = {}         
    for record in same_location:
        encoding_value = pgh.encode(float(record[0]), float(record[1]), precision= 5)  
        if encoding_value in geo_dictionary:
            geo_dictionary[encoding_value].append(record)
        else:
            geo_dictionary[encoding_value] = [record]
    
    for key in geo_dictionary:
        if len(geo_dictionary[key]) == 1:
            fire_value = geo_dictionary[key][0]
            if float(climate_value[2]) > 20 and float(climate_value[7]) > 180:
            
                # make a record out of the climate and single fire data value
                new_record = { "latitude" : float(fire_value[0]),
                 "longitude": float(fire_value[1]),
                 "datetime" : fire_value[5],
                 "confidence" : float(fire_value[2]),
                 "surface_temperature_celsius" : float(fire_value[3]),
                "date": fire_value[4],
                "air_temperature_celsius" : float(climate_value[2]),
                "relative_humidity": float(climate_value[3]),
                "windspeed_knots" : float(climate_value[4]),
                "max_wind_speed": float(climate_value[5]),
                "precipitation_level": get_prec_number(climate_value[6]),
                "precipitation_type": get_prec_letter(climate_value[6]),
                "GHI_w/m2": float(climate_value[7]),
                "cause": 'natural'}
                the_collection.insert(new_record)
                
                
            else:
               # make a record out of the climate and single fire data value
            
                new_record = { "latitude" : float(fire_value[0]),
                 "longitude": float(fire_value[1]),
                 "datetime" : fire_value[5],
                 "confidence" : float(fire_value[2]),
                 "surface_temperature_celsius" : float(fire_value[3]),
                "date": fire_value[4],
                "air_temperature_celsius" : float(climate_value[2]),
                "relative_humidity": float(climate_value[3]),
                "windspeed_knots" : float(climate_value[4]),
                "max_wind_speed": float(climate_value[5]),
                "precipitation_level": get_prec_number(climate_value[6]),
                "precipitation_type": get_prec_letter(climate_value[6]),
                "GHI_w/m2": float(climate_value[7]),
                "cause": 'other'}
                the_collection.insert(new_record)                
                
        else:
            #two fires happened at least
            fire_numbs = geo_dictionary[key]  # this is a list of records
            possible_averages = []
            non_averages = []
            for fires in fire_numbs:
                for other_fires in fire_numbs:
                    #times are the same
                    matched = False
                    if fires != other_fires and fires[5] == other_fires[5]:
                        possible_averages.append(fires)
                        possible_averages.append(other_fires)
                        matched = True
                if matched == False:  # then you went through and found nothing with the same time
                    non_averages.append(fires)
            if possible_averages:            
                average_confidence = 0
                average_sur_temp = 0
                # make a record out of the climate data and the average of everything else.
                first_record = possible_averages[0]
                for record in possible_averages:
                    average_confidence += float(record[2])
                    average_sur_temp += float(record[3])
                average_confidence = average_confidence / len(fire_numbs)
                average_sur_temp = average_sur_temp / len(fire_numbs)

                if float(climate_value[2]) > 20 and float(climate_value[7]) > 180:
                    new_record = { "latitude" : float(climate_value[0]),
                     "longitude": float(climate_value[1]),
                     "datetime" : first_record[5],
                     "confidence" : average_confidence,
                     "surface_temperature_celsius" : average_sur_temp,
                    "date": climate_value[8],
                    "air_temperature_celsius" : float(climate_value[2]),
                    "relative_humidity": float(climate_value[3]),
                    "windspeed_knots" : float(climate_value[4]),
                    "max_wind_speed": float(climate_value[5]),
                    "precipitation_level": get_prec_number(climate_value[6]),
                    "precipitation_type": get_prec_letter(climate_value[6]),
                    "GHI_w/m2": float(climate_value[7]),
                    "cause": 'natural'}
                    the_collection.insert(new_record)

                else:
                    new_record = { "latitude" : float(climate_value[0]),
                     "longitude": float(climate_value[1]),
                     "datetime" : first_record[5],
                     "confidence" : average_confidence,
                     "surface_temperature_celsius" : average_sur_temp,
                    "date": climate_value[8],
                    "air_temperature_celsius" : float(climate_value[2]),
                    "relative_humidity": float(climate_value[3]),
                    "windspeed_knots" : float(climate_value[4]),
                    "max_wind_speed": float(climate_value[5]),
                    "precipitation_level": get_prec_number(climate_value[6]),
                    "precipitation_type": get_prec_letter(climate_value[6]),
                    "GHI_w/m2": float(climate_value[7]),
                    "cause": 'other'}
                    the_collection.insert(new_record)
                    
            else:
                for record in non_averages:
                    if float(climate_value[2]) > 20 and float(climate_value[7]) > 180:
                        new_record = { "latitude" : float(climate_value[0]),
                         "longitude": float(climate_value[1]),
                         "datetime" : record[5],
                         "confidence" : average_confidence,
                         "surface_temperature_celsius" : average_sur_temp,
                        "date": climate_value[8],
                        "air_temperature_celsius" : float(climate_value[2]),
                        "relative_humidity": float(climate_value[3]),
                        "windspeed_knots" : float(climate_value[4]),
                        "max_wind_speed": float(climate_value[5]),
                        "precipitation_level": get_prec_number(climate_value[6]),
                        "precipitation_type": get_prec_letter(climate_value[6]),
                        "GHI_w/m2": float(climate_value[7]),
                        "cause": 'natural'}
                        the_collection.insert(new_record)

                    else:
                        new_record = { "latitude" : float(climate_value[0]),
                         "longitude": float(climate_value[1]),
                         "datetime" : record[5],
                         "confidence" : average_confidence,
                         "surface_temperature_celsius" : average_sur_temp,
                        "date": climate_value[8],
                        "air_temperature_celsius" : float(climate_value[2]),
                        "relative_humidity": float(climate_value[3]),
                        "windspeed_knots" : float(climate_value[4]),
                        "max_wind_speed": float(climate_value[5]),
                        "precipitation_level": get_prec_number(climate_value[6]),
                        "precipitation_type": get_prec_letter(climate_value[6]),
                        "GHI_w/m2": float(climate_value[7]),
                        "cause": 'other'}
                        the_collection.insert(new_record)
            

            
    if geo_dictionary == {}:
        # insert the climate data straight away but with no fire records
        new_record = {
            "latitude" : float(climate_value[0]),
            "longitude": float(climate_value[1]),
             "datetime": climate_value[9],
            "confidence" : None,
            "surface_temperature_celsius": None,
            "date": climate_value[8],
            "air_temperature_celsius" : float(climate_value[2]),
            "relative_humidity": float(climate_value[3]),
            "windspeed_knots" : float(climate_value[4]),
            "max_wind_speed": float(climate_value[5]),
            "precipitation_level": get_prec_number(climate_value[6]),
            "precipitation_type": get_prec_letter(climate_value[6]),
            "GHI_w/m2": float(climate_value[7]),
             "cause": 'other'
        }
        the_collection.insert(new_record)  
    client.close()

n_secs = 10
topic = "same_topic"

conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
sc = SparkContext.getOrCreate()

sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'week11-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary
#kafkaStream.pprint()
lines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendDataToDB))

ssc.start()
time.sleep(600) # Run stream for 10 minutes just in case no detection of producer
# ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)

KeyboardInterrupt: 

In [4]:
client = MongoClient() # so we connect on the default host and port
db = client.fit3182_assignment_db # create a new database fit3182_assignment_db
the_collection = db.climate  # create a new collection called 'the_collection'
for k in the_collection.find({}):
    pprint(k)

{'GHI_w/m2': 134.0,
 '_id': ObjectId('60a9cebdbe3498736e23d129'),
 'air_temperature_celsius': 15.0,
 'cause': 'other',
 'confidence': None,
 'date': '2019/01/12',
 'datetime': '00:00:00',
 'latitude': -37.608,
 'longitude': 149.292,
 'max_wind_speed': 12.0,
 'precipitation_level': 0.02,
 'precipitation_type': 'G',
 'relative_humidity': 44.6,
 'surface_temperature_celsius': None,
 'windspeed_knots': 9.7}
{'GHI_w/m2': 102.0,
 '_id': ObjectId('60a9cec2be3498736e23d12b'),
 'air_temperature_celsius': 11.0,
 'cause': 'other',
 'confidence': 85.0,
 'date': '2019/01/16',
 'datetime': '04:48:00',
 'latitude': -37.3655,
 'longitude': 148.2821,
 'max_wind_speed': 26.0,
 'precipitation_level': 0.0,
 'precipitation_type': 'G',
 'relative_humidity': 40.4,
 'surface_temperature_celsius': 59.0,
 'windspeed_knots': 19.3}
{'GHI_w/m2': 104.0,
 '_id': ObjectId('60a9ceccbe3498736e23d12d'),
 'air_temperature_celsius': 12.0,
 'cause': 'other',
 'confidence': None,
 'date': '2019/01/14',
 'datetime': '00:00:0